In [1]:
import os
os.listdir('/home/student/dbanalysis/dbanalysis/resources')

['trimmed_routes.json',
 '.ipynb_checkpoints',
 'Scraping api.ipynb',
 'scrape_bus_schedule_API.py',
 'stops.js',
 'trimmed_dep_times.json',
 'cleanweather.csv',
 'reduced-traffic-map.js',
 'Bus_stops_schedule.json',
 'heatmap.pickle',
 '__pycache__',
 'models',
 'clean_routes.pickle',
 'trimmed_stops_shapes_map.pickle',
 'officalroutes.pickle',
 'stops_trimmed.json',
 'schedule_keys.pickle',
 'dep_times.pickle']

## Deciding on 'foot distance' between stops, and choosing the number of 'foot' links to involve

In [3]:
import json
stops = json.loads(open('/home/student/dbanalysis/dbanalysis/resources/stops_trimmed.json','r').read())

In [4]:
import haversine


In [5]:
stops.keys()

dict_keys(['7612', '2', '3', '4', '6', '7', '8', '10', '11', '12', '14', '15', '16', '17', '18', '19', '21', '22', '23', '24', '25', '27', '28', '29', '30', '31', '32', '33', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '63', '64', '65', '66', '68', '69', '71', '72', '73', '74', '75', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '90', '91', '92', '93', '94', '102', '104', '105', '107', '110', '112', '113', '114', '115', '116', '117', '118', '119', '126', '127', '128', '129', '130', '131', '132', '133', '134', '136', '137', '138', '139', '140', '141', '142', '143', '146', '147', '148', '149', '150', '151', '153', '154', '155', '156', '157', '158', '159', '164', '165', '166', '167', '168', '169', '173', '174', '175', '176', '177', '178', '179', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '19

In [9]:
import time
t1=time.time()
stop = '7612'
for stop in [s for s in stops.keys() if s != '7612']:
    haversine.haversine((stops['7612']['lat'],stops['7612']['lon']),(stops[stop]['lat'],stops[stop]['lon']))
print(t1-time.time())

-0.021638154983520508


In [ ]:
#Ok so, so timewise it isn't too bad.

In [13]:
t1=time.time()
links = {}
for stopA in stops:
    links[stopA]=[]
    for stopB in stops:
        
        if stopA != stopB:
            
            links[stopA].append(tuple([stopB,haversine.haversine((stops[stopA]['lat'],stops[stopA]['lon']),(stops[stopB]['lat'],stops[stopB]['lon']))]))
            
            
print(time.time()-t1)
        

116.00716662406921


So two questions --> how many links per stop and what is a sufficient walking distance between stops?

This does give us some hope for running djikstra's algorithm on the graph though. That there was 25000000 calculations and it didn't take that long really.

### What if we say 

In [15]:
from operator import itemgetter
for key in links:
    
    links[key]=sorted(links[key],key=itemgetter(1))

In [16]:
import pickle
with open('/home/student/dbanalysis/dbanalysis/resources/stop_foot_distance.pickle','wb') as handle:
    pickle.dump(links,handle)

In [17]:
print(hello)

NameError: name 'hello' is not defined

actually this file is just too big to justify. I think we should cut it down to twenty for this iteration

In [23]:
g={}
for key in links:
    
    g[key]=links[key][:5]

In [24]:
with open('/home/student/dbanalysis/dbanalysis/resources/stop_foot_distance.pickle','wb') as handle:
    pickle.dump(g,handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
g['7612']

[('408', 0.05380943260694801),
 ('2810', 0.0660497365787014),
 ('494', 0.07282352928472953),
 ('2905', 0.15481241695119913),
 ('2809', 0.19299465808620164)]

In [ ]:
#Actually - why not threshold it at five. Fuck it. Ho

# Attempting djikstra

In [29]:
print('hello')

hello


In [17]:
#attempting djikstra
from dbanalysis.network import linear_network
from dbanalysis.classes import time_tabler
#import time
g=linear_network.bus_network()
for n in g.nodes:
    g.nodes[n].get_foot_links()
    g.nodes[n].all_links = set([i for i in g.nodes[n].get_links() if i in g.nodes] + [i for i in g.nodes[n].foot_links if i in g.nodes])

In [3]:
for n in g.nodes:
    g.nodes[n].get_foot_links()
    g.nodes[n].all_links = set([i for i in g.nodes[n].get_links() if i in g.nodes] + [i for i in g.nodes[n].foot_links if i in g.nodes])

In [34]:
visited = []
unvisited = []
current_time = 30000
stop = '246'
links = set([i for i in g.nodes['246'].get_links() if i in g.nodes] + [i[0] for i in g.nodes['246'].foot_links if i in g.nodes])
print(links)

{'4', '248', '2'}


In [2]:
for n in g.nodes:
    g.nodes[n].all_links = set([i for i in g.nodes[n].get_links() if i in g.nodes] + [i[0] for i in g.nodes[n].foot_links if i in g.nodes])

In [38]:
g.nodes['2'].all_links

{'192', '3', '4', '4726', '4962', '6', '7'}

In [29]:
from math import inf
for n in g.nodes:
    g.nodes[n].weight = inf

In [12]:


from math import inf
for n in g.nodes:
    g.nodes[n].weight = inf
    g.nodes[n].back_links = []
current_node = '182'
g.nodes['182'].weight = 30000
current_time = 30000
visited = []
to_visit=[current_node]
count = 0
import time as timeeee
t1=timeeee.time()
while len(to_visit) > 0 and current_node != '2847':
    if count % 1000 == 0:
        print(count)
    count+=1
    
    node = g.nodes[current_node]
    links = g.nodes[current_node].all_links
    for link in links:
        
        if link in g.nodes[current_node].get_links() and hasattr(g.nodes[current_node].timetable,'data'):
            try:
                time = g.nodes[current_node].timetable.get_next_departure(link,current_time)
                
                if time < g.nodes[link].weight:
                    g.nodes[link].weight = time
                    g.nodes[link].back_links.append(current_node)
                    if link not in to_visit:
                        to_visit.append(link)
            except:
                pass
        elif link in g.nodes[current_node].foot_links:
            time = g.nodes[current_node].foot_links[link] + current_time
            if time < g.nodes[link].weight:
                g.nodes[link].weight = time
                g.nodes[link].back_links.append(current_node)
                if link not in to_visit:
                    to_visit.append(link)
                
    to_visit.remove(current_node)
    minimum = inf
    current_node = None
    for node in to_visit:
        if g.nodes[node].weight < minimum:
            minimum = g.nodes[node].weight
            current_node = node
            current_time = g.nodes[node].weight
            
print(timeeee.time()-t1)
print(current_node,current_time)
            
            
            
            
    
    
    
    


0
1000
2000
8.831852912902832
2847 35355.63696433865


In [12]:
len(g.nodes)

4595

In [16]:
g.nodes.keys()

dict_keys(['7612', '2', '3', '4', '6', '7', '8', '10', '11', '12', '14', '15', '16', '17', '18', '19', '21', '22', '23', '24', '25', '27', '28', '29', '30', '31', '32', '33', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '63', '64', '65', '66', '68', '69', '71', '72', '73', '74', '75', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '90', '91', '92', '93', '94', '102', '104', '105', '107', '110', '112', '113', '114', '115', '116', '117', '118', '119', '126', '127', '128', '129', '130', '131', '132', '133', '134', '136', '137', '138', '139', '140', '141', '142', '143', '146', '147', '148', '149', '150', '151', '153', '154', '155', '156', '157', '158', '159', '164', '165', '166', '167', '168', '169', '173', '174', '175', '176', '177', '178', '179', '181', '182', '183', '184', '185', '186', '187', '188', '189', '190', '191', '192', '193', '194', '195', '196', '19

In [13]:
weight = g.nodes['2847'].weight
cur_node = '2847'
while weight > 30000:
    minweight = inf
    for link in g.nodes[cur_node].back_links:
        if g.nodes[link].weight < minweight:
            minweight=g.nodes[link].weight
            new_curnode = link
    print(stops_dict[str(new_curnode)], weight)
    weight = minweight
    cur_node = new_curnode

{'stop_name': 'Ballinteer, Ballinteer Avenue', 'lat': 53.27548878928, 'lon': -6.25164594920425} 35355.63696433865
{'stop_name': 'Ballinteer, Broadford Road (Broadford Rise)', 'lat': 53.27605941543899, 'lon': -6.25501160402915} 35344.40658614399
{'stop_name': 'Ballinteer, Broadford Lawn', 'lat': 53.27783035506061, 'lon': -6.25502943056719} 35176.930471320724
{'stop_name': 'Ballinteer, Broadford Road (Chestnut Grove)', 'lat': 53.2791896181841, 'lon': -6.25516900458426} 35097.84773388106
{'stop_name': 'Ballinteer, Broadford Walk', 'lat': 53.2808192504807, 'lon': -6.25843194409367} 35043.14458398487
{'stop_name': 'Broadford Road (Stone Masons Way)', 'lat': 53.28081152749479, 'lon': -6.26098174584064} 34987.49406331329
{'stop_name': 'Nutgrove, Llewellyn Way', 'lat': 53.282920437969295, 'lon': -6.2632207525028} 34939.33504362357
{'stop_name': 'Rathfarnham, Broadford Road (Dargle View)', 'lat': 53.28381862036179, 'lon': -6.26502908175177} 34844.94134997793
{'stop_name': 'Nutgrove, Marley Cour

 Ok... well, whatever. Whatever. Whatever...

In [5]:
import json
stops_dict = json.loads(open('/home/student/dbanalysis/dbanalysis/resources/stops_trimmed.json','r').read())

In [18]:
def djikstra(departure,dep_time,destination):
    global g
    from math import inf
    for n in g.nodes:
        g.nodes[n].weight = inf
        g.nodes[n].back_links = []
    current_node = departure
    g.nodes[departure].weight = 30000
    current_time = dep_time
    visited = []
    to_visit=[current_node]
    count = 0
    import time as timeeee
    t1=timeeee.time()
    while len(to_visit) > 0 and current_node != destination:
        if count % 1000 == 0:
            print(count)
        count+=1

        node = g.nodes[current_node]
        links = g.nodes[current_node].all_links
        for link in links:

            if link in g.nodes[current_node].get_links() and hasattr(g.nodes[current_node].timetable,'data'):
                try:
                    time = g.nodes[current_node].timetable.get_next_departure(link,current_time)

                    if time < g.nodes[link].weight:
                        g.nodes[link].weight = time
                        g.nodes[link].back_links.append(current_node)
                        if link not in to_visit:
                            to_visit.append(link)
                except:
                    pass
            elif link in g.nodes[current_node].foot_links:
                time = g.nodes[current_node].foot_links[link] + current_time
                if time < g.nodes[link].weight:
                    g.nodes[link].weight = time
                    g.nodes[link].back_links.append(current_node)
                    if link not in to_visit:
                        to_visit.append(link)

        to_visit.remove(current_node)
        minimum = inf
        current_node = None
        for node in to_visit:
            if g.nodes[node].weight < minimum:
                minimum = g.nodes[node].weight
                current_node = node
                current_time = g.nodes[node].weight
        
        
    weight = g.nodes[destination].weight
    cur_node = destination
    while weight > dep_time:
        minweight = inf
        for link in g.nodes[cur_node].back_links:
            if g.nodes[link].weight < minweight:
                minweight=g.nodes[link].weight
                new_curnode = link
        print(stops_dict[str(new_curnode)], weight)
        weight = minweight
        cur_node = new_curnode

                
    

In [21]:
djikstra('2892',30000,'3062')

0
1000
2000
{'stop_name': 'Glasthule, Buckley Galleries', 'lat': 53.2859751405739, 'lon': -6.1163170447788895} 36395.79494592841
{'stop_name': 'Glasthule, Brighton House', 'lat': 53.286086994700206, 'lon': -6.1194169517962305} 36179.132526947025
{'stop_name': 'Glasthule, Glasthule Road', 'lat': 53.2869934492863, 'lon': -6.12275201988687} 36030.49562093205
{'stop_name': 'Glasthule, Glasthule Church', 'lat': 53.2877232595891, 'lon': -6.125719965456541} 35855.15260963781
{'stop_name': 'Glasthule, Summerhill Road', 'lat': 53.2884237752038, 'lon': -6.127399294125731} 35702.553529674646
{'stop_name': "Dun Laoghaire, Upper George's Street", 'lat': 53.29017029107779, 'lon': -6.1310130392851905} 35604.54855118659
{'stop_name': 'Dun Laoghaire, Royal Marine Road', 'lat': 53.2927382320774, 'lon': -6.1354313618542005} 35382.7446912208
{'stop_name': 'Dun Laoghaire, Bloomfield Shopping Centre', 'lat': 53.293625487561904, 'lon': -6.1392931718791} 35075.37949890022
{'stop_name': 'Dun Laoghaire, Vesey P